
# Demo Backtest: اثر شاخص CPI روی قیمت بیت‌کوین

> هدف: نشان دادن رویکرد اولیه‌ی آزمون رابطه‌ی بین تورم (CPI) و بازده BTC.


In [ ]:
# شبیه‌سازی داده‌ها (بدون اتصال واقعی به FRED)
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import datetime as dt

np.random.seed(42)
dates = pd.date_range("2020-01-01", periods=48, freq="M")

# CPI YoY شبیه‌سازی‌شده (%)
cpi_yoy = pd.Series(2 + np.random.randn(len(dates))*0.5, index=dates).abs()

# قیمت BTC شبیه‌سازی‌شده
btc_price = pd.Series(8000 + np.cumsum(np.random.randn(len(dates))*1000), index=dates).abs()

df = pd.DataFrame({"cpi_yoy": cpi_yoy, "btc_price": btc_price})
df["btc_return"] = df["btc_price"].pct_change()

df.head()



## تحلیل همبستگی


In [ ]:
corr = df["cpi_yoy"].corr(df["btc_return"])
corr



## ترسیم سری‌ها


In [ ]:
plt.figure()
df["cpi_yoy"].plot(title="CPI YoY (simulated)")
plt.show()

plt.figure()
df["btc_price"].plot(title="BTC Price (simulated)")
plt.show()



## بک‌تست ساده: استراتژی شرطی بر CPI
- اگر CPI YoY > 3% → از BTC خارج شو (ریسک بالاتر).
- در غیر این صورت → سرمایه‌گذاری در BTC.


In [ ]:
signal = df["cpi_yoy"] <= 3
df["strategy_return"] = df["btc_return"] * signal.shift(1).fillna(False)

cum_btc = (1 + df["btc_return"].fillna(0)).cumprod()
cum_strategy = (1 + df["strategy_return"].fillna(0)).cumprod()

plt.figure()
cum_btc.plot(label="Buy & Hold BTC")
cum_strategy.plot(label="CPI-based Strategy")
plt.legend()
plt.title("Backtest (simulated)")
plt.show()



## نکات عملیاتی
- داده‌ی واقعی CPI از FRED (سری CPIAUCSL) و BTC از صرافی‌ها/aggregators بارگیری شود.
- تست‌های جدی نیازمند lag consideration، out-of-sample، و مقایسه با benchmark است.
